In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr


In [ ]:
train = pd.read_csv('train.csv')

In [ ]:
test = pd.read_csv('test.csv')

In [ ]:
train .corr().iloc[-1].sort_values(ascending=False)

In [ ]:
str_columns = train.drop(train.corr().index.to_list(), axis=1).columns.to_list()
train_ohe = train.copy()
map_dict = {}

for column in str_columns:
    encoded, labels = pd.factorize(train[column])
    train_ohe[column] = encoded
    map_dict[column] = labels
    
train_ohe = train_df_ohe.fillna(0)

In [ ]:
test_ohe = test.copy()

for column in str_columns:
    remap_dict = dict(enumerate(map_dict[column]))
    remap_dict[-1] = np.nan
    remap_dict = {value: key for key, value in remap_dict.items()}
    
    test_ohe[column] = [remap_dict[label] for label in test[column]]

test_ohe = test_ohe.fillna(0)

In [ ]:
rf = RandomForestRegressor()
parameters  = {'n_estimators': range(10, 51, 10),
               'max_depth': range(1, 13, 1),
               'min_samples_leaf': range(1, 8),
               'min_samples_split': range(2, 10, 2)}
clf = GridSearchCV(estimator=rf, param_grid=parameters, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
rf_best = clf.best_estimator_
clf.best_params_

In [ ]:
clf.best_score_

In [ ]:
feature_importance = pd.DataFrame({
    'feature': list(X_train),
    'importance': rf_best.feature_importances_
    }).sort_values(by='importance', ascending=False)
feature_importance.T

In [ ]:
X_test_final = test_ohe[X.columns]
y_forecast_final = pd.Series(rf_best.predict(X_test_final))

In [ ]:
results = (pd.concat([test_df_ohe.Id, y_forecast_final], axis=1)
           .rename(columns={0: 'SalePrice'}))
results